In [1]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService
from cc3d.core.PyCoreSpecs import PottsCore, CellTypePlugin, VolumePlugin, ContactPlugin, BlobInitializer
from cc3d.core.PySteppables import SteppableBasePy
from cc3d.core.GraphicsUtils.JupyterGraphicsFrameWidget import JupyterGraphicsFrameClient
from IPython.display import display

# Define simulation lattice and neighborhood
potts_specs = PottsCore(dim_x=100, dim_y=100, neighbor_order=2)

# Define cell types
cell_type_specs = CellTypePlugin("Condensing", "NonCondensing")

# Volume constraint for both cell types
volume_specs = VolumePlugin()
volume_specs.param_new("Condensing", target_volume=25, lambda_volume=2)
volume_specs.param_new("NonCondensing", target_volume=25, lambda_volume=2)

# Adhesion (contact energy) between types
contact_specs = ContactPlugin(neighbor_order=2)
contact_specs.param_new(type_1="Medium", type_2="Condensing", energy=20)
contact_specs.param_new(type_1="Medium", type_2="NonCondensing", energy=20)
contact_specs.param_new(type_1="Condensing", type_2="Condensing", energy=2)
contact_specs.param_new(type_1="Condensing", type_2="NonCondensing", energy=11)
contact_specs.param_new(type_1="NonCondensing", type_2="NonCondensing", energy=16)

# Initialize cells as a blob
blob_init_specs = BlobInitializer()
blob_init_specs.region_new(width=5, radius=20, center=(50, 50, 0), cell_types=("Condensing", "NonCondensing"))

# Define a custom steppable to implement cell growth logic
class GrowthSteppable(SteppableBasePy):
    def start(self):
        # Set initial volume parameters for each cell
        for cell in self.cell_list:
            cell.targetVolume = 25
            cell.lambdaVolume = 2

    def step(self, mcs):
        # Example: update target volume every 100 steps
        if mcs % 100 == 0:
            for cell in self.cell_list:
                cell.targetVolume += 1  # simple growth

# Create a simulation service instance
cc3d_sim = CC3DSimService()

# Register all the specifications with the simulation service
cc3d_sim.register_specs([potts_specs, cell_type_specs, volume_specs, contact_specs, blob_init_specs])

# Register the custom steppable with the simulation service
cc3d_sim.register_steppable(steppable=GrowthSteppable, frequency=1)

# Run the simulation
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

# Create a JupyterGraphicsFrameClient for visualization
graphics_frame = JupyterGraphicsFrameClient()
graphics_frame.launch()  # Launch the visualization frame
graphics_frame.show()  # Display the visualization frame

# Run the simulation for a specified number of steps
num_steps = 1000
while cc3d_sim.current_step < num_steps:
    cc3d_sim.step()
    graphics_frame.draw()  # Update the visualization


/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizes
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizesFloating
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: RecentSimulations
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
Random number generator: MersenneTwister


WILL RUN SIMULATION FROM BEGINNING


CC3DViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=600)